In [82]:
from instagram.client import InstagramAPI

api = InstagramAPI(
    access_token='TOKEN',
    client_secret='SECRET')

def get_liked_media():
    media = []
    liked_media, next_ = api.user_liked_media()
    media.extend(liked_media)
    while next_:
        liked_media, next_ = api.user_liked_media(with_next_url=next_)
        media.extend(liked_media)
        print("Remaining API Calls: %s/%s" % (api.x_ratelimit_remaining, api.x_ratelimit))
    return media

In [83]:
media = get_liked_media()

Remaining API Calls: 4998/5000
Remaining API Calls: 4997/5000
Remaining API Calls: 4996/5000
Remaining API Calls: 4995/5000
Remaining API Calls: 4994/5000
Remaining API Calls: 4993/5000
Remaining API Calls: 4992/5000
Remaining API Calls: 4991/5000
Remaining API Calls: 4990/5000
Remaining API Calls: 4989/5000
Remaining API Calls: 4988/5000


## Display liked images

In [ ]:
from IPython.display import display, HTML

for item in media:
    display(HTML(('<img src="%s"/>' % item.get_standard_resolution_url())))
    
    

In [84]:
from functools import reduce
from collections import Counter

def retreive_tags(result, tags):
    result += list(map(lambda tag: tag.name, tags))
    return result

tags_collections = (map(lambda photo: photo.tags, media))
tags_collections = reduce(retreive_tags, tags_collections, [])

top_10_tags_with_count = Counter(tags_collections).most_common(10)
top_10_tags = list(map(lambda tag: tag[0], top_10_tags_with_count))
top_10_tags_with_count

[('igerscz', 42),
 ('vscocam', 35),
 ('iglifecz', 35),
 ('vsco', 26),
 ('prague', 22),
 ('livefolk', 21),
 ('liveauthentic', 20),
 ('vscocze', 18),
 ('czech', 14),
 ('czech_vibes', 14)]

In [24]:
def get_media_for_tag(tag):
    media = []
    tag_recent_media, next_ = api.tag_recent_media(tag_name=tag)
    media.extend(tag_recent_media)
    while next_ and len(media) < 10:
        tag_recent_media, next_ = api.tag_recent_media(tag_name=tag, with_next_url=next_)
        media.extend(tag_recent_media)
        print("Remaining API Calls: %s/%s" % (api.x_ratelimit_remaining, api.x_ratelimit))
    return media

In [85]:
from itertools import groupby

media_for_user = []
for tag in top_10_tags:
    print("loading... {0}".format(tag))
    media_for_user.extend(get_media_for_tag(tag))
    

loading... igerscz
loading... vscocam
loading... iglifecz
loading... vsco
loading... prague
loading... livefolk
loading... liveauthentic
loading... vscocze
loading... czech
loading... czech_vibes


We've got all the photos. We just need to pick the most liked ones.

Some weighting would be great. Number of occurences of top 10 tags related to number of likes should be enough.

In [110]:
from itertools import islice

def unique(seq, idfun=None):
   if idfun is None:
       def idfun(x): return x
   seen = {}
   result = []
   for item in seq:
       marker = idfun(item)
       if marker in seen: continue
       seen[marker] = 1
       result.append(item)
   return result


media_for_user_processed = map(lambda m: {'id': m.id, 'author': m.user.username, 'caption': m.caption, 'likes': m.like_count, 'link': m.link, 'url': m.get_standard_resolution_url(), 'tags': map(lambda t: t.name, m.tags)}, media_for_user)
media_for_user_processed = unique(media_for_user_processed, lambda m: m['id'])


def weight_by_tags(tags):
    weight = 1
    for tag in tags:
        tag_weight = [w[1] for w in top_10_tags_with_count if w[0] == tag]
        weight += (1 if len(tag_weight) == 0 else tag_weight[0]) * 0.01
    return weight

media_for_user_processed = sorted(media_for_user_processed, key = lambda m: m['likes'] * weight_by_tags(m['tags']), reverse = True)
media_for_user_processed = islice(media_for_user_processed, 5)
list(media_for_user_processed)


[{'author': 'vojtafaltyn',
  'caption': Comment: vojtafaltyn said "🙌🏻 Mirror 🙌🏻 #vsconature #wonderful_places #lifeofadventure #streetdreamsmag #natureaddict #big_shotz #master_shots #instagoodmyphoto #visualsoflife #liveauthentic #livefolk #letsgosomewhere #instamagazine_ #igerscz #iglifecz #vscostyle #mobilemag #artofvisuals #vscocze #exks_nature #earthpix #topvsco #vscovibe #exploretocreate #czech #XIXtagram #folkgood #exklusive_shot #czech_vibes #jj_mextures",
  'id': '1108752537369395166_380118534',
  'likes': 196,
  'link': 'https://instagram.com/p/9jFI7ttz_e/',
  'tags': <map at 0x10696e1d0>,
  'url': 'https://scontent.cdninstagram.com/hphotos-xaf1/t51.2885-15/s640x640/sh0.08/e35/12142387_1520544921594002_2099671531_n.jpg'},
 {'author': 'marek.paci',
  'caption': Comment: marek.paci said "S u p e r b  A u t u m n a l  D a y //🌞🌲🍁
  #autmn #autumnlover #sunsetXmary #necekamziju
  Dnešní den byl  hodně vydařenej , jako i ten včerejší! 🔆 a tak si tohohle počasí je potřeba užít! 🍁
 